In [26]:
# default_exp convert_to_zarr

# Notebook which converts per region netCDF files to Zarr files to make them more efficient when indexing

### uses pangeo_small environment

In [1]:
#test_ignore
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#export
import xarray as xr
import zarr
from joblib import Parallel, delayed
import pandas as pd
import os

## Set the parameters here
### Ensure all State and Regions you want to transform are specified here. 

In [3]:
#test_ignore
data_root = '/media/scottcha/E1/Data/OAPMLData/'

interpolation = 1

#currently only have Washington regions and one season specified for the tutorial
#uncomment regions and seasons if doing a larger transform
regions = {#'Utah': ['Abajos', 'Logan', 'Moab', 'Ogden', 'Provo', 
           #'Salt Lake', 'Skyline', 'Uintas'],  
           #'Colorado': ['Grand Mesa Zone', 'Sangre de Cristo Range', 'Steamboat Zone', 'Front Range Zone',
           #'Vail Summit Zone', 'Sawatch Zone', 'Aspen Zone', 
           #'North San Juan Mountains', 'South San Juan Mountains', 'Gunnison Zone'],
           'Washington': ['Mt Hood', 'Olympics']#, 'Snoqualmie Pass', 'Stevens Pass',
           #'WA Cascades East, Central', 'WA Cascades East, North', 'WA Cascades East, South',
           #'WA Cascades West, Central', 'WA Cascades West, Mt Baker', 'WA Cascades West, South'
           #]
           }
seasons = ['18-19'] #['15-16', '16-17', '17-18', '18-19']

In [4]:
#export
class ConvertToZarr:
    """
    Class which encapsulates the logic to convert a set of filtered netCDF files to Zarr
    """
    
    def __init__(self, seasons, regions, data_root, interpolate=1):
        """
        Initialize the class
        
        Keyword Arguments
        seasons: list of season values to process
        regions: dictonary of Key: State and Value: List of Regions to process for that state
        data_root: the root path of the data folders which contains the 3.GFSFiltered1xInterpolation
        interpolate: the amount of interpolation applied in in the previous ParseGFS notebook (used for finding the correct input/output paths)
        """
        self.processed_path = data_root + '/3.GFSFiltered'+ str(interpolate) + 'xInterpolation/'
        self.zarr_base_path = data_root + '/4.GFSFiltered'+ str(interpolate) + 'xInterpolationZarr/'
        
        self.seasons = seasons
        self.regions = regions
        self.data_root = data_root
        
        if not os.path.exists(self.zarr_base_path):
            os.makedirs(self.zarr_base_path)
    
    def compute_region(self, region_name, season, state):
        """
        Calculates the zarr conversion for a specific region, season and state and indexes it for efficient lookup 
        
        Keyword Arguments
        region_name: name of the region to process
        season: season to process
        state: state to process (region must be a part of the state)
        """
        first = True
        base_path = self.processed_path + season + '/' + '/Region_' + region_name 
        zarr_path = self.zarr_base_path + season + '/' + state + '/Region_' + region_name + '.zarr'
        
        #TODO: refactor these to be shared code as logic also exists in ParseGFS
        p = 181
        if season in ['15-16', '19-20']:
            p = 182 #leap years

        snow_start_date = '2015-11-01'
        if season == '16-17':
            snow_start_date = '2016-11-01'
        if season == '17-18':
            snow_start_date = '2017-11-01'
        if season == '18-19':
            snow_start_date = '2018-11-01'
        if season == '19-20':
            snow_start_date = '2019-11-01'

        date_values_pd = pd.date_range(snow_start_date, periods=p, freq="D")
        try:
            with xr.open_zarr(zarr_path) as z:
                if z.time.values[-1] == date_values_pd[-1]:
                    print(' already exists: ' + region_name + ' ' + season + ' ' + state)
                    z.close()
                    return
                else:
                    #already exists but incomplete
                    date_values_pd = [pd.Timestamp(v) for v in date_values_pd.values.astype('datetime64[ns]') if v not in z.time.values]
                    print(' some exist but have to complete ' + str(len(date_values_pd)))
                    first = False
        except ValueError as err:
            #ignore as it doesn't exist yet
            pass
        
        #sometimes vars get added, filter to only the list of vars in the first dataset for that region
        #TODO: handle the case where the first dataset has more vars than subsequent ones
        final_vars = None
        for d in date_values_pd:

            path =  base_path + '_' + d.strftime('%Y%m%d') + '.nc'
            print('On ' + str(path.split('/')[-1]))

            try:
                ds = xr.open_dataset(path, chunks={'latitude':1, 'longitude':1})
            except OSError as err:
                print(' missing file: ' + path)
                continue

            ds = ds.to_array(name='vars').chunk({'time':1, 'latitude':1, 'longitude':1, 'variable':-1}).to_dataset()

            try:

                if first:
                    final_vars = list(ds.variable.values)
                    ds.to_zarr(zarr_path, consolidated=True)
                    first=False
                else:
                    assert(final_vars is not None)
                    ds = ds.sel(variable=ds.variable.isin(final_vars))
                    ds.to_zarr(zarr_path, consolidated=True, append_dim='time')
            except ValueError as err:
                print('Value Error ' + format(err) + ' on ' + zarr_path )                
                return

    def process_tuple(self, t): 
        """
        Entry method to call compute_region with a tuple
        Basically a helper for executing with joblib parallel
        
        Keyword Arguments
        t: the tuple containing the region, season and state
        """
        self.compute_region(t[0], t[1], t[2])
    
    def make_list(self):
        """
        Helper method to make the list of values to process
        """
        to_process = []
        for s in self.seasons:
            for state in self.regions.keys():           
                for r in self.regions[state]:
                    to_process.append((r,s,state))
        return to_process
    
    def convert_local(self, jobs=15):
        l = self.make_list()
    
        #one state & season takes about 6 hours with 15 cores on my machine
        Parallel(n_jobs=jobs, backend="multiprocessing")(map(delayed(self.process_tuple), l))

In [5]:
#tmppath =  '/media/scottcha/E1/Data/OAPMLData/3.GFSFiltered1xInterpolation/19-20/Region_Mt Hood_20191120.nc'
#ds = xr.open_dataset(tmppath, chunks={'latitude':1, 'longitude':1})
#ds = ds.to_array(name='vars').chunk({'time':1, 'latitude':1, 'longitude':1, 'variable':-1}).to_dataset()

In [6]:
#final_vars = list(ds.variable.values)

In [7]:
#tmp_ds = ds.sel(variable=ds.variable.isin(final_vars))

In [8]:
#ds

In [9]:
#tmp_ds

In [10]:
#test_ignore
ctz = ConvertToZarr(seasons, regions, data_root)

In [11]:
#test_ignore
ctz.convert_local()

On Region_Mt Hood_20181101.ncOn Region_Olympics_20181101.nc

On Region_Mt Hood_20181102.nc
On Region_Mt Hood_20181103.nc
On Region_Mt Hood_20181104.nc
On Region_Mt Hood_20181105.nc
On Region_Olympics_20181102.nc
On Region_Mt Hood_20181106.nc
On Region_Mt Hood_20181107.nc
On Region_Mt Hood_20181108.nc
On Region_Mt Hood_20181109.nc
On Region_Mt Hood_20181110.nc
On Region_Olympics_20181103.nc
On Region_Mt Hood_20181111.nc
On Region_Mt Hood_20181112.nc
On Region_Mt Hood_20181113.nc
On Region_Mt Hood_20181114.nc
On Region_Mt Hood_20181115.nc
On Region_Olympics_20181104.nc
On Region_Mt Hood_20181116.nc
On Region_Mt Hood_20181117.nc
On Region_Mt Hood_20181118.nc
On Region_Mt Hood_20181119.nc
On Region_Mt Hood_20181120.nc
On Region_Olympics_20181105.nc
On Region_Mt Hood_20181121.nc
On Region_Mt Hood_20181122.nc
On Region_Mt Hood_20181123.nc
On Region_Mt Hood_20181124.nc
On Region_Olympics_20181106.nc
On Region_Mt Hood_20181125.nc
On Region_Mt Hood_20181126.nc
On Region_Mt Hood_20181127.nc
On R

On Region_Olympics_20190129.nc
On Region_Olympics_20190130.nc
On Region_Olympics_20190131.nc
On Region_Olympics_20190201.nc
On Region_Olympics_20190202.nc
On Region_Olympics_20190203.nc
On Region_Olympics_20190204.nc
On Region_Olympics_20190205.nc
On Region_Olympics_20190206.nc
On Region_Olympics_20190207.nc
On Region_Olympics_20190208.nc
On Region_Olympics_20190209.nc
On Region_Olympics_20190210.nc
On Region_Olympics_20190211.nc
On Region_Olympics_20190212.nc
On Region_Olympics_20190213.nc
On Region_Olympics_20190214.nc
On Region_Olympics_20190215.nc
On Region_Olympics_20190216.nc
On Region_Olympics_20190217.nc
On Region_Olympics_20190218.nc
On Region_Olympics_20190219.nc
On Region_Olympics_20190220.nc
On Region_Olympics_20190221.nc
On Region_Olympics_20190222.nc
On Region_Olympics_20190223.nc
On Region_Olympics_20190224.nc
On Region_Olympics_20190225.nc
On Region_Olympics_20190226.nc
On Region_Olympics_20190227.nc
On Region_Olympics_20190228.nc
On Region_Olympics_20190301.nc
On Regio